In [5]:
%%writefile prog.cu

#include <stdio.h>
#include <cuda_runtime.h>

__global__ void atomicKernel(int* shared_counter)
{
  // Each thread tries tp increment the shared shared_counter
  atomicAdd(shared_counter,1);
  *shared_counter++;
}

int main()
{
    int h_counter = 0;
    int* d_counter;

    cudaMalloc((void**)&d_counter, sizeof(int));
    cudaMemcpy(d_counter, &h_counter, sizeof(int), cudaMemcpyHostToDevice);

    atomicKernel<<<1, 1024>>> (d_counter);

    cudaDeviceSynchronize();

    cudaMemcpy(&h_counter, d_counter, sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_counter);

    printf("Final counter value: %d\n", h_counter);

    return 0;
}

Writing prog.cu


In [6]:
!nvcc -o exe prog.cu
!./exe

Final counter value: 1024


In [1]:
%%writefile prog1.cu

#include <stdio.h>
#include <cuda_runtime.h>
#define N 1024

__global__ void atomicKernel(int *shared_counter, int *d_array)
{
  int idx = blockIdx.x*blockDim.x + threadIdx.x;

  if (idx < N)
      {
      atomicAdd(shared_counter, d_array[idx]);
      }
//  *shared_counter++;
}



int main()
{
    int h_array[N];
    int *d_array;
    int *d_result;
    int h_result;

    for (int i=0;i<N;i++)
    {
      h_array[i]=i+1;
    }

    cudaMalloc(&d_array, N*sizeof(int));
    cudaMemcpy(d_array, h_array, N*sizeof(int), cudaMemcpyHostToDevice);

    cudaMalloc(&d_result, sizeof(int));

    atomicKernel<<<1, 1024>>> (d_result, d_array);

    cudaDeviceSynchronize();

    cudaMemcpy(&h_result, d_result, sizeof(int), cudaMemcpyDeviceToHost);

    printf("Final value: %d\n", h_result);

    cudaFree(d_array);
    cudaFree(d_result);

    return 0;
}

Writing prog1.cu


In [2]:
!nvcc -o exe prog1.cu
!./exe

Final value: 524800


In [3]:
%%writefile prog2.cu

#include <stdio.h>
#include <cuda_runtime.h>
#define N 1024

__global__ void atomicKernel(int *a, int *result)
{
    *result = (*a * *a + 1)/2;
}



int main()
{
    int h_var;
    int *d_var;
    int *d_res;
    int h_res;

    h_var=N;

    cudaMalloc(&d_var, sizeof(int));
    cudaMemcpy(d_var, &h_var, sizeof(int), cudaMemcpyHostToDevice);

    cudaMalloc(&d_res, sizeof(int));

    atomicKernel<<<1, 1>>> (d_var, d_res);

    cudaDeviceSynchronize();

    cudaMemcpy(&h_res, d_res, sizeof(int), cudaMemcpyDeviceToHost);

    printf("Final value: %d\n", h_res);

    cudaFree(d_var);
    cudaFree(d_res);

    return 0;
}

Overwriting prog2.cu


In [4]:
!nvcc -o exe prog2.cu
!./exe

Final value: 524288


3) Find the minimum/maximum in an array without allowing for race conditions to occur:

atomicMax(max_val, arr[idx]);